In [2]:
# Importacion de librerias
import pandas as pd
import numpy as np
import json
import re
import gzip
import ast
import warnings
from textblob import TextBlob
warnings.filterwarnings('ignore')

In [9]:
# Importar users_items
rows = []
with gzip.open('./assets/users_items.json.gz', 'rt', encoding='utf8') as f:
    print("importando users_items...")
    for line in f.readlines():        
        data_dic = ast.literal_eval(line)
        user_id = data_dic['user_id']
        # items_count = data_dic['items_count']
        steam_id = data_dic['steam_id']
        # user_url = data_dic['user_url']
        items_list = data_dic['items']
        data_list = []
        for items in items_list:
            data = {
                'user_id': user_id, 
                # items_count, 
                'steam_id': steam_id
                #, user_url
            }     
            for key, value in items.items():
                data.update({key:value})
            rows.append(data)
# print("insertamos los datos en un DataFrame...")
users_items = pd.DataFrame(rows)
users_items.to_csv('./assets/users_items.csv', index=False)

importando users_items...


KeyboardInterrupt: 

In [99]:
# Importar users_reviews
rows = []
with gzip.open('./assets/user_reviews.json.gz', 'rt', encoding='utf8') as f:
    print("importando user_reviews...")
    for line in f.readlines():        
        data_dic = ast.literal_eval(line)
        user_id = data_dic['user_id']
        # user_url = data_dic['user_url']
        reviews_list = data_dic['reviews']
        data_list = []
        for reviews in reviews_list:
            # analisis de sentimientos
            if reviews["review"] == "":
                sentiment_analysis = 1
            else:
                review_text = reviews["review"]
                if review_text == "":
                    sentiment_analysis = 1
                else:
                    blob = TextBlob(review_text)
                    sentiment_polarity = blob.sentiment.polarity
                    if sentiment_polarity < -0.1:
                        sentiment_analysis = 0
                    elif sentiment_polarity > 0.1:
                        sentiment_analysis = 2
                    else:
                        sentiment_analysis = 1
            data = {
                'user_id': user_id, 
                # user_url,
                'recommend': reviews["recommend"],
                'sentiment': sentiment_analysis,
                'item_id': reviews['item_id'],
                'review': reviews['review'],
            }  
            # agregar todos los campos de reviews   
            # for key, value in reviews.items():
            #     data.update({key:value})
            rows.append(data)
users_reviews = pd.DataFrame(rows)
users_reviews.to_csv('./assets/users_reviews.csv', index=False)

importando user_reviews...


In [ ]:
# Importar steam_games
years = range(1950, 2024)
rows = []
with gzip.open('./assets/steam_games.json.gz', 'rt', encoding='utf8') as f:
    print("importando steam_games...")
    for line in f.readlines():
        # Lee el archivo JSON
        data_json = json.loads(line)
        # Convierte el JSON en un diccionario
        data_dic = dict(data_json)
        # si id es NaN termino la ejecución
        if str(data_dic['id']) != "nan":
            data={
                'publisher': data_dic['publisher'],
                'genres': data_dic['genres'],
                'app_name': data_dic['app_name'],
                'release_date': data_dic['release_date'],
                'price': data_dic['price'],
                'item_id': data_dic['id'],
            }
            for key, value in data_dic.items():        
                # si el precio no es float lo reemplazo por 0
                if key == "price": 
                    if isinstance(value, float) and value > 0:
                        data.update({key: value})
                    else:
                        data.update({key: 0})
                # si el año no esta en el rango lo reemplazo por None
                if key == "release_date":
                    if not isinstance(value, str):
                        data.update({key:None})                     
                    else:
                        # Buscar el año dentro de la cadena de texto utilizando una expresión regular
                        patron = r"\b(19[5-9]\d|20\d{2})\b"
                        match = re.search(patron, value)
                        # Si se encuentra un año dentro del rango, reemplazarlo por el mismo
                        if match:
                            year = int(match.group(0))
                            if year in years:
                                data.update({key:year})
                            else:
                                data.update({key:None})
                        else:
                            data.update({key:None})
                # si esta en data y es nan lo reemplazo por None
                if key in data and str(value).find("nan") != -1:
                    data.update({key:None})
            rows.append(data)
steam_games = pd.DataFrame(rows)
steam_games.to_csv('./assets/steam_games.csv', index=False)

In [100]:
# Importar users_items de csv
print("importando users_items...")
users_items = pd.read_csv('./assets/users_items.csv')
# Importar users_reviews de csv
print("importando users_reviews...")
users_reviews = pd.read_csv('./assets/users_reviews.csv')
# Importar steam_games de csv
print("importando steam_games...")
steam_games = pd.read_csv('./assets/steam_games.csv')
# uno los tres dataframes
print("unificando los dataframes...")
df = pd.merge(steam_games, users_items, on=['item_id'], how='inner')
df = pd.merge(df, users_reviews, on=['user_id'], how='inner')
# guardo el dataframe en un archivo csv
print("guardando el dataframe en un archivo csv...")
df.to_csv('./assets/steam.csv', index=False)
print("termino la importación!!!")

importando users_items...
importando users_reviews...
importando steam_games...
unificando los dataframes...
guardando el dataframe en un archivo csv...
termino la importación!!!


In [5]:
# Importar steam
print("importando steam...")
data = pd.read_csv("./assets/steam.csv.gz", compression='gzip')
# Mostrar los primeros 5 registros
print("Mostrar los primeros 5 registros...")
print(data.head())


importando steam...
Mostrar los primeros 5 registros...
             publisher                         genres              app_name  \
0  Stainless Games Ltd  ['Action', 'Indie', 'Racing']  Carmageddon Max Pack   
1  Stainless Games Ltd  ['Action', 'Indie', 'Racing']  Carmageddon Max Pack   
2  Stainless Games Ltd  ['Action', 'Indie', 'Racing']  Carmageddon Max Pack   
3  Stainless Games Ltd  ['Action', 'Indie', 'Racing']  Carmageddon Max Pack   
4  Stainless Games Ltd  ['Action', 'Indie', 'Racing']  Carmageddon Max Pack   

   release_date  price  item_id_x   user_id           steam_id  \
0        1997.0   9.99     282010  UTNerd24  76561198053985682   
1        1997.0   9.99     282010  UTNerd24  76561198053985682   
2        1997.0   9.99     282010  UTNerd24  76561198053985682   
3        1997.0   9.99     282010  UTNerd24  76561198053985682   
4        1997.0   9.99     282010  UTNerd24  76561198053985682   

              item_name  playtime_forever  playtime_2weeks  recommend  \

In [27]:
# API
def PlayTimeGenre(genre: str):
    # Filtramos por genero
    data_genres = data.dropna(subset=['genres'])
    data_genres = data_genres[data_genres['genres'].str.contains(genre)]
    # Agrupamos por año y sumamos las horas jugadas
    data_genres = data_genres.groupby('release_date')['playtime_forever'].sum().reset_index()
    # Obtenemos el año con mayor horas jugadas
    year = int(data_genres[data_genres['playtime_forever'] == data_genres['playtime_forever'].max()]['release_date'].values[0])    
    return {'Año de lanzamiento con más horas jugadas para Género': genre, 'Año': year}
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para Género': 'Action', 'Año': 2012}

In [71]:
def UserForGenre(genre: str):
    # Eliminar generos nulos
    data_genres = data.dropna(subset=['genres'])
    # Eliminar usuarios nulos
    data_genres = data_genres.dropna(subset=['user_id'])
    # Filtramos por genero
    data_genres = data_genres[data_genres['genres'].str.contains(genre)]
    # Agrupamos por usuario y sumamos las horas jugadas
    data_playtime = data_genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    # Obtenemos el usuario con mayor horas jugadas
    user = data_playtime[data_playtime['playtime_forever'] == data_playtime['playtime_forever'].max()]['user_id'].values[0]
    # Filtramos por usuario
    data_user = data_genres[data_genres['user_id'] == user]
    # Agrupamos por año y sumamos las horas jugadas
    data_year = data_user.groupby('release_date')['playtime_forever'].sum().reset_index()
    years = data_year.to_dict('records')
    # Obtenemos el año con mayor horas jugadas
    year = int(data_genres[data_genres['playtime_forever'] == data_genres['playtime_forever'].max()]['release_date'].values[0])
    print(f"'Usuario con más horas jugadas para Género {genre}': {user}, 'Horas jugadas': {years}")
UserForGenre('Action')

'Usuario con más horas jugadas para Género Action': Sp3ctre, 'Horas jugadas': [{'release_date': 1993.0, 'playtime_forever': 0}, {'release_date': 1995.0, 'playtime_forever': 1736}, {'release_date': 1996.0, 'playtime_forever': 0}, {'release_date': 1998.0, 'playtime_forever': 0}, {'release_date': 1999.0, 'playtime_forever': 352}, {'release_date': 2000.0, 'playtime_forever': 565152}, {'release_date': 2001.0, 'playtime_forever': 104}, {'release_date': 2002.0, 'playtime_forever': 1904}, {'release_date': 2003.0, 'playtime_forever': 61384}, {'release_date': 2004.0, 'playtime_forever': 1019288}, {'release_date': 2005.0, 'playtime_forever': 170712}, {'release_date': 2006.0, 'playtime_forever': 7168}, {'release_date': 2007.0, 'playtime_forever': 902272}, {'release_date': 2008.0, 'playtime_forever': 1792}, {'release_date': 2009.0, 'playtime_forever': 866608}, {'release_date': 2010.0, 'playtime_forever': 624664}, {'release_date': 2011.0, 'playtime_forever': 1239168}, {'release_date': 2012.0, 'playt

In [90]:
# def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
# Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
def UsersRecommend(year: int):
    # Filtramos por año
    data_year = data[data['release_date'] == year]
    # Filtramos por recomendados
    data_year = data_year[data_year['recommend'] == True]
    # Filtramos por comentarios positivos/neutrales
    data_year = data_year[data_year['sentiment'] > 0]
    # Agrupamos por juego y contamos las recomendaciones
    data_year = data_year.groupby('app_name')['recommend'].count().reset_index()
    # Ordenamos de mayor a menor
    data_year = data_year.sort_values(by='recommend', ascending=False)
    # Obtenemos el top 3
    top3 = data_year.head(3).to_dict('records')
    return {'Top 3 de juegos MÁS recomendados por usuarios para el año': year, 'Top 3': top3}
UsersRecommend(2010)


{'Top 3 de juegos MÁS recomendados por usuarios para el año': 2010,
 'Top 3': [{'app_name': 'Just Cause 2', 'recommend': 16988},
  {'app_name': "Sid Meier's Civilization® V", 'recommend': 14115},
  {'app_name': 'Fallout: New Vegas', 'recommend': 13521}]}

In [91]:
# def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
# Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
def UsersNotRecommend(year: int):
    # Filtramos por año
    data_year = data[data['release_date'] == year]
    # Filtramos por no recomendados
    data_year = data_year[data_year['recommend'] == False]
    # Filtramos por comentarios negativos
    data_year = data_year[data_year['sentiment'] == 0]
    # Agrupamos por juego y contamos las recomendaciones
    data_year = data_year.groupby('app_name')['recommend'].count().reset_index()
    # Ordenamos de mayor a menor
    data_year = data_year.sort_values(by='recommend', ascending=False)
    # Obtenemos el top 3
    top3 = data_year.head(3).to_dict('records')
    return {'Top 3 de juegos MENOS recomendados por usuarios para el año': year, 'Top 3': top3}
UsersNotRecommend(2010)

{'Top 3 de juegos MENOS recomendados por usuarios para el año': 2010,
 'Top 3': [{'app_name': 'Just Cause 2', 'recommend': 1030},
  {'app_name': 'Arma 2: Operation Arrowhead', 'recommend': 821},
  {'app_name': "Sid Meier's Civilization® V", 'recommend': 793}]}

In [108]:
# def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
# Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}
def sentiment_analysis(year: int):
    # Filtramos por año
    data_year = data[data['release_date'] == year]
    # Agrupamos por sentimiento y contamos las reseñas
    data_year = data_year.groupby('sentiment')['review'].count().reset_index()
    # Obtenemos el top 3
    sentiment = data_year.to_dict('records')
    # Inicializar contadores
    negative_count = 0
    neutral_count = 0
    positive_count = 0
    # Contar el número de reseñas con cada sentimiento
    for s in sentiment:
        if s['sentiment'] == 0:
            negative_count += s['review']
        elif s['sentiment'] == 1:
            neutral_count += s['review']
        elif s['sentiment'] == 2:
            positive_count += s['review']
    # Crear el diccionario con los contadores
    sentiment = {'Negative': negative_count, 'Neutral': neutral_count, 'Positive': positive_count}
    return {'Según el año de lanzamiento': year, 'Sentimiento': sentiment}
sentiment_analysis(2010)

{'Según el año de lanzamiento': 2010,
 'Sentimiento': {'Negative': 55083, 'Neutral': 164486, 'Positive': 168501}}